# Phase 1

In [61]:
import os
import pandas as pd
import re
import streamlit as st
import nltk
import joblib

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
joblib.dump(lr, "sentiment_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from collections import Counter


ModuleNotFoundError: No module named 'streamlit'

In [7]:
from google.colab import files
files.upload()

Saving data.csv to data (1).csv


In [12]:
os.listdir("/content")

['.config', 'data (1).csv', 'data.csv', 'sample_data']

In [16]:
df = pd.read_csv("data.csv")
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [17]:
df.columns

Index(['Reviewer Name', 'Review Title', 'Place of Review', 'Up Votes',
       'Down Votes', 'Month', 'Review text', 'Ratings'],
      dtype='object')

In [21]:
df.rename(columns={
    'Review text': 'review_text',
    'Ratings': 'rating'
}, inplace=True)

df.head()


/tmp/ipython-input-2180835062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={


,review_text,rating
0,"Nice product, good quality, but price is now r...",4
1,They didn't supplied Yonex Mavis 350. Outside ...,1
2,Worst product. Damaged shuttlecocks packed in ...,1
3,"Quite O. K. , but nowadays the quality of the...",3
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [22]:
df = df[['review_text', 'rating']]
df.head()

,review_text,rating
0,"Nice product, good quality, but price is now r...",4
1,They didn't supplied Yonex Mavis 350. Outside ...,1
2,Worst product. Damaged shuttlecocks packed in ...,1
3,"Quite O. K. , but nowadays the quality of the...",3
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [23]:
df.isnull().sum()
df.dropna(inplace=True)

In [25]:
df = df[df['rating'] != 3]

In [26]:
df['sentiment'] = df['rating'].apply(
    lambda x: 'positive' if x >= 4 else 'negative'
)

df.head()

/tmp/ipython-input-2851011921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['rating'].apply(


,review_text,rating,sentiment
0,"Nice product, good quality, but price is now r...",4,positive
1,They didn't supplied Yonex Mavis 350. Outside ...,1,negative
2,Worst product. Damaged shuttlecocks packed in ...,1,negative
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1,negative
5,Good quality product. Delivered on time.READ MORE,5,positive


# Phase 2

In [29]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [30]:
def clean_text(text):
    # convert to lowercase
    text = text.lower()

    # remove special characters & numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # tokenize and remove stopwords + lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word)
             for word in words
             if word not in stop_words]

    return " ".join(words)


In [53]:
df['clean_review'] = df['review_text'].apply(clean_text)
df.head()

,review_text,rating,sentiment,clean_review
0,"Nice product, good quality, but price is now r...",4,positive,nice product good quality price rising bad sig...
1,They didn't supplied Yonex Mavis 350. Outside ...,1,negative,didnt supplied yonex mavis outside cover yonex...
2,Worst product. Damaged shuttlecocks packed in ...,1,negative,worst product damaged shuttlecock packed new b...
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1,negative,pricedjust retaileri didnt understand wat adva...
5,Good quality product. Delivered on time.READ MORE,5,positive,good quality product delivered timeread


In [32]:
df[['review_text', 'clean_review']].head(3)

,review_text,clean_review
0,"Nice product, good quality, but price is now r...",nice product good quality price rising bad sig...
1,They didn't supplied Yonex Mavis 350. Outside ...,didnt supplied yonex mavis outside cover yonex...
2,Worst product. Damaged shuttlecocks packed in ...,worst product damaged shuttlecock packed new b...


# Phase 3

In [33]:
X = df['clean_review']
y = df['sentiment']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [38]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [39]:
X_train_tfidf.shape, X_test_tfidf.shape

((6316, 5000), (1579, 5000))

# Phase 4

In [44]:
lr = LogisticRegression(max_iter=1000)

lr.fit(X_train_tfidf, y_train)

y_pred_lr = lr.predict(X_test_tfidf)

f1_lr = f1_score(y_test, y_pred_lr, pos_label='positive')
print("Logistic Regression F1-score:", f1_lr)

Logistic Regression F1-score: 0.954239091876552


In [45]:
nb = MultinomialNB()

nb.fit(X_train_tfidf, y_train)

y_pred_nb = nb.predict(X_test_tfidf)

f1_nb = f1_score(y_test, y_pred_nb, pos_label='positive')
print("Naive Bayes F1-score:", f1_nb)

Naive Bayes F1-score: 0.9488795518207283


In [46]:
print("Logistic Regression Report:\n")
print(classification_report(y_test, y_pred_lr))

Logistic Regression Report:

              precision    recall  f1-score   support

    negative       0.84      0.49      0.62       214
    positive       0.93      0.99      0.95      1365

    accuracy                           0.92      1579
   macro avg       0.88      0.74      0.79      1579
weighted avg       0.91      0.92      0.91      1579



In [47]:
print("Naive Bayes Report:\n")
print(classification_report(y_test, y_pred_nb))

Naive Bayes Report:

              precision    recall  f1-score   support

    negative       0.89      0.36      0.52       214
    positive       0.91      0.99      0.95      1365

    accuracy                           0.91      1579
   macro avg       0.90      0.68      0.73      1579
weighted avg       0.91      0.91      0.89      1579



# Phase 5 : Already done in header (Saved Best Module)

# Phase 6

In [51]:
negative_reviews = df[df['sentiment'] == 'negative']
negative_reviews.head()

,review_text,rating,sentiment,clean_review
1,They didn't supplied Yonex Mavis 350. Outside ...,1,negative,didnt supplied yonex mavis outside cover yonex...
2,Worst product. Damaged shuttlecocks packed in ...,1,negative,worst product damaged shuttlecock packed new b...
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1,negative,pricedjust retaileri didnt understand wat adva...
13,This is the second time i am getting an opened...,1,negative,second time getting opened bundle yonex mavis ...
16,I usually buy them the same model but this did...,1,negative,usually buy model last even dayread


In [54]:
all_words = " ".join(negative_reviews['clean_review']).split()
word_freq = Counter(all_words)

word_freq.most_common(20)

[('shuttle', 300),
 ('quality', 254),
 ('product', 214),
 ('bad', 178),
 ('qualityread', 122),
 ('good', 117),
 ('worst', 102),
 ('poor', 89),
 ('goodread', 88),
 ('productread', 79),
 ('one', 72),
 ('dont', 71),
 ('buy', 63),
 ('day', 59),
 ('box', 55),
 ('mavis', 52),
 ('badread', 52),
 ('read', 50),
 ('cork', 50),
 ('time', 46)]

In [56]:
bigrams = list(ngrams(all_words, 2))
Counter(bigrams).most_common(15)

[(('poor', 'quality'), 37),
 (('bad', 'quality'), 36),
 (('dont', 'buy'), 34),
 (('poor', 'qualityread'), 34),
 (('bad', 'qualityread'), 31),
 (('quality', 'shuttle'), 30),
 (('quality', 'goodread'), 23),
 (('worst', 'product'), 22),
 (('qualityread', 'bad'), 21),
 (('bad', 'product'), 19),
 (('bad', 'productread'), 17),
 (('quality', 'good'), 16),
 (('waste', 'moneyread'), 16),
 (('yonex', 'mavis'), 14),
 (('low', 'quality'), 13)]

In [62]:
!streamlit run app.py

/bin/bash: line 1: streamlit: command not found
